# Open Power System Data: Renewable Energy Power Plant List


## Part 1: Download and Processing the Original Data

This script downlads and extracts the original data from the sources and merges them in a single data
frame. It subsequently adds the geolocation for each power plant and gives an overview over the
structure of the data frame. Finally it saves the data frame to a SQLite file.

# Table of contents 

* [1. Script Setup](#1.-Script-Setup)
* [2. Download and Data Extraction](#2.-Download-and-Data-Extraction)
    * [2.1 Download and Create Individual Data Frames](##-2.1-Download-and-Create-Individual-Data-Frames)
    * [2.2 Definition of Data Frame Structure](##-2.2-Definition-of-Data-Frame-Structure)
    * [2.3 Merge Data Frames](##2.3-Merge-Data-Frames)
    * [2.4 Translation and Summary](##2.4-Translation-and-Summary)
* [3 Georeferencing](#3-Georeferencing)
    * [3.1 Get Coordinates by Postcode](##-3.1-Get-Coordinates-by-Postcode)
    * [3.2 Transform Geoinformation](##3.2-Transform-Geoinformation)




# 1. Script Setup


In [1]:
# importing all necessary Python libraries for this Script
#%matplotlib inline

from collections import OrderedDict
import io
import json
import os
import subprocess
import zipfile
import posixpath
import urllib.parse
import urllib.request
import numpy as np
import pandas as pd
# not in use
# import requests
import datetime  
import sqlite3 
import utm
import logging

# Set up a log
logger = logging.getLogger('notebook')
logger.setLevel('INFO')
nb_root_logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s'\
                              '- %(message)s',datefmt='%d %b %Y %H:%M:%S')
nb_root_logger.handlers[0].setFormatter(formatter)

# Create input and output folders if they don't exist
os.makedirs('input/original_data', exist_ok=True)
os.makedirs('output', exist_ok=True)
os.makedirs('output/datapackage_renewables', exist_ok=True)

# 2. Download and Data Extraction

The data which will be processed below is provided by the following sources:

**1. [Netztransparenz.de](https://www.netztransparenz.de/de/Anlagenstammdaten.htm)** - Official grid transparency platform from the German TSOs (50Hertz, Amprion, TenneT and TransnetBW).

**2. Bundesnetzagentur (BNetzA)** - German Federal Network Agency for Electricity, Gas, Telecommunications, Posts and Railway (Data for [roof-mounted PV power plants](http://www.bundesnetzagentur.de/cln_1422/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/ErneuerbareEnergien/Photovoltaik/DatenMeldgn_EEG-VergSaetze/DatenMeldgn_EEG-VergSaetze_node.html) and for [all other renewable energy power plants](http://www.bundesnetzagentur.de/cln_1412/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/ErneuerbareEnergien/Anlagenregister/Anlagenregister_Veroeffentlichung/Anlagenregister_Veroeffentlichungen_node.html))

## 2.1 Download and Create Individual Data Frames

**Specify the source URLs:**

In [3]:
# point URLs to original data
url_netztransparenz ='https://www.netztransparenz.de/de/file/'\
                  'Anlagenstammdaten_2014_4UeNB.zip'

url_bnetza ='http://www.bundesnetzagentur.de/SharedDocs/Downloads/DE/Sachgebiete/'\
            'Energie/Unternehmen_Institutionen/ErneuerbareEnergien/Anlagenregister/'\
            'VOeFF_Anlagenregister/2016_04_Veroeff_AnlReg.xls?__blob=publicationFile&v=2'
        
url_bnetza_pv = 'https://www.bundesnetzagentur.de/SharedDocs/Downloads/DE/'\
                'Sachgebiete/Energie/Unternehmen_Institutionen/'\
                'ErneuerbareEnergien/Photovoltaik/Datenmeldungen/'\
                'Meldungen_Aug-Mrz2016.xls?__blob=publicationFile&v=2'

**Download Data:**

In [4]:
def download_and_cache(url):
    """This function downloads a file into a folder called 
    original_data and returns the local filepath."""
    
    path = urllib.parse.urlsplit(url).path
    filename = posixpath.basename(path)
    filepath = "input/original_data/"+filename
    
    #check if file exists, if not download it
    if  not os.path.exists(filepath):
        print("Downloading file", filename)
        urllib.request.urlretrieve(url, filepath)
    else:
        print("Using local file from", filepath)
    filepath = ''+filepath
    return filepath

**Unzip and load downloaded data as data frames**

To process the provided data [pandas DataFrame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) is applied.<br>
*This section of data loading takes approx. 5-15 min. *

In [5]:
# load zip file for data from netztransparenz.de
z = zipfile.ZipFile(download_and_cache(url_netztransparenz))


# Get TSO data from zip file
amprion_df = pd.read_csv(z.open('Amprion_Anlagenstammdaten_2014.csv'),
                         sep=';',
                         thousands='.',
                         decimal=',',
                         header=0,
                         parse_dates=[11, 12, 13, 14],
                         encoding='cp850',
                         dayfirst=True,
                         low_memory=False)

hertz_df = pd.read_csv(z.open('50Hertz_Anlagenstammdaten_2014.csv'),
                       sep=';',
                       thousands='.',
                       decimal=',',
                       header=0,
                       parse_dates=[11, 12, 13, 14],
                       encoding='cp1252',
                       dayfirst=True,
                       low_memory=False)

tennet_df = pd.read_csv(z.open('TenneT_Anlagenstammdaten_2014.csv'),
                        sep=';',
                        thousands='.',
                        decimal=',',
                        header=0,
                        parse_dates=[11, 12, 13, 14],
                        encoding='cp1252',
                        dayfirst=True,
                        low_memory=False)

transnetbw_df = pd.read_csv(z.open('TransnetBW_Anlagenstammdaten_2014.csv'),
                            sep=';',
                            thousands='.',
                            decimal=',',
                            header=0,
                            parse_dates=[11, 12, 13, 14],
                            encoding='cp1252',
                            dayfirst=True,
                            low_memory=False)

print('Download and cache bnetz_a')
# Get BNetzA register 
bnetza_df = pd.read_excel(download_and_cache(url_bnetza),
                          sheetname='Gesamtübersicht',
                          header=0,
                          converters={'4.9 Postleit-zahl': str})

# Get BNetzA-PV register
bnetza_pv = pd.ExcelFile(download_and_cache(url_bnetza_pv))


# Combine all PV BNetzA sheets into one data frame
bnetza_pv_df = pd.concat(bnetza_pv.parse(sheet, skiprows=10,
                                         converters={'Anlage \nPLZ': str}
                                         ) for sheet in bnetza_pv.sheet_names)

# Drop not needed NULL "Unnamed:" column
bnetza_pv_df = bnetza_pv_df.drop(bnetza_pv_df.columns[[7]], axis=1)

Download and cache bnetz_a


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:645)>

## 2.2 Definition of Data Frame Structure

**Translation**<br>
To standardise the data frame the original column names from the German TSOs and the BNetzA wil be translated and new english column names wil be assigned to the data frame. The unique column names are needed in order to merge the data frames together.<br>
The translation list is provided here: **[csv](https://github.com/Open-Power-System-Data/datapackage_renewable_power_plants/blob/master/input/column_translation_list.csv).**

In [ ]:
# Get translation list
columnnames = pd.read_csv('input/column_translation_list.csv', sep=",",
                          header=0)

columndict = columnnames.set_index('original_name')['column_naming'].to_dict()

# Translate columns by list 
bnetza_pv_df.rename(columns=columndict, inplace=True)
bnetza_df.rename(columns=columndict, inplace=True)
transnetbw_df.rename(columns=columndict, inplace=True)
tennet_df.rename(columns=columndict, inplace=True)
amprion_df.rename(columns=columndict, inplace=True)
hertz_df.rename(columns=columndict, inplace=True)

# Translate special cases separately
backslash = {'Anlage \nBundesland': 'federal_state', 'Anlage \nOrt oder Gemarkung':
             'city', 'Anlage \nPLZ': 'postcode', 'Anlage \nStraße oder Flurstück *)':
             'address', 'Installierte \nNennleistung [kWp]': 'electrical_capacity'}

bnetza_pv_df.rename(columns = backslash, inplace=True)

**Add source names and generation types**<br>
In a second step all source names and (for the BNetzA-PV data) the generation types will be added separately to the data frame.

In [ ]:
# Add source names to the data frames
transnetbw_df['source'] = 'TransnetBW'
tennet_df['source'] = 'TenneT'
amprion_df['source'] = 'Amprion'
hertz_df['source'] = '50Hertz'
bnetza_df['source'] = 'BNetzA'
bnetza_pv_df['source'] = 'BNetzA_PV'

# Add for the BNetzA PV data the generation types
bnetza_pv_df['generation_type'] = 'solar'
bnetza_pv_df['generation_subtype'] = 'solar_roof_mounted'

## 2.3 Merge Data Frames

The individual data frames from the TSOs (Netztransparenz.de) and BNetzA will be merged together.<br>
The column *generation_subtype* will be filled with values from the Column *generation_type* (it wil be used in the next step 2.4 to differentiate between generation types and subtypes).<br>
In the last step of this section only the columns of interest will be selected and the final data frame created.

**Merge individual data frames**

In [ ]:
# Merge data frames
dataframes = [transnetbw_df, tennet_df, amprion_df, hertz_df, bnetza_pv_df, bnetza_df]

renewables = pd.concat(dataframes)

renewables = renewables.reset_index()

**Fill column "generation_subtype"**

In [ ]:
# Transfer generation_type values to generation_subtype
idx_subtype = renewables[(renewables['source'] != 'BNetzA_PV')].index

renewables['generation_subtype'].loc[idx_subtype] = (renewables['generation_type'].loc[idx_subtype])

**Select columns of our interest**

In this section only the columns of interest for the final data frame are selected and the rest discarded. 

In [ ]:
column_interest = ['start_up_date', 'electrical_capacity', 'generation_type',
                   'generation_subtype', 'thermal_capacity', 'city', 'postcode',
                   'address', 'tso', 'dso', 'utm_zone', 'utm_east', 'utm_north',
                   'notification_reason', 'eeg_id',
                   'voltage_level', 'decommission_date',
                   'power_plant_id', 'source']

In [ ]:
renewables = renewables.loc[:, column_interest]
renewables.reset_index(drop=True)
logger.info('Clean dataframe from not needed columns')

**First look at the final data frame structure and format**

`DataFrame.info()` shows us the number of non-null (non-NA) values in each column, which can serve as a first indicator.

In [ ]:
renewables.info()

## 2.4 Translation and Summary

In this section the different German terms for generation types, subtypes and voltage levels will be translated and harmonized across the individual data sources (see [value_translation_list.csv](https://github.com/Open-Power-System-Data/datapackage_renewable_power_plants/blob/master/input/value_translation_list.csv)).

Additionally the different classifications and terms for generation types and subtypes from the individual sources will be assigned accordign to the following table (see [generation_types_translation_list.csv](https://github.com/Open-Power-System-Data/datapackage_renewable_power_plants/blob/master/input/generation_types_translation_list.csv)):

|generation_type|generation_subtype|
|---|---|
|wind|wind_onshore|
| |wind_offshore|
|solar|solar_ground_mounted|
||solar_roof_mounted|
|gas|gas|
||gas_sewage|
||gas_landfill|
||gas_mine|
|biomass|biomass|
||biofuel|
||biogas|
||biogas_from_grid|
||biogas_dry_fermentation|
||wood|
||waste_wood|
|hydro|hydro|
|geothermal|geothermal|

**Translation**

In [ ]:
# Read translation list
translation_list = pd.read_csv('input/value_translation_list.csv', 
                               sep=",",
                               header=0)
# Create dictionnary in order to change values 
translation_dict = translation_list.set_index('original_name')['opsd_naming'].to_dict()

postcode = pd.read_csv('input/de_tso_postcode_gps.csv',
                       sep=';',
                       header=0)

types_dict = types_list.set_index('generation_subtype')['generation_type'].to_dict()

**Separate and assign generation types and subtypes**

In [ ]:
# Running time ~ 10 min
renewables.replace(translation_dict, inplace=True)

# Create new column for generation_subtype
renewables['generation_subtype'] = renewables.generation_type

# Replace generation_subtype by generation_type
renewables.generation_type.replace(types_dict, inplace=True)

**Summary of data frame**

In [ ]:
# Electrical capacity per generation type (in MW)
renewables.groupby(['generation_type'])['electrical_capacity'].sum() / 1000

In [ ]:
# Electrical capacity per generation subtype (in MW)
renewables.groupby(['generation_subtype'])['electrical_capacity'].sum() / 1000

# 3 Georeferencing
## 3.1 Get Coordinates by Postcode
*(for data with no existing geocoordinates)*

The available post code in the original data provides a first approximation for the geocoordinates of the RE power plants.<br>
The BNetzA data provides the full zip code whereas due to data privacy the TSOs only report the first three digits of the power plant's post code (e.g. 024xx) and no address. Subsequently a centroid of the post code region polygon is used to find the coordinates.

With data from
*  http://www.suche-postleitzahl.org/downloads?download=plz-gebiete.shp.zip
*  http://www.suche-postleitzahl.org/downloads?download_file=plz-3stellig.shp.zip
*  http://www.suche-postleitzahl.org/downloads

a CSV-file for all existing German post codes with matching geocoordinates has been compiled. The latitude and longitude coordinates were generated by running a PostgreSQL + PostGIS database. Additionally the respective TSO has been added to each post code. *(A Link to the SQL script will follow here later)*

*(License: http://www.suche-postleitzahl.org/downloads, Open Database Licence for free use. Source of data: © OpenStreetMap contributors)*

In [ ]:
# Read generated postcode/location file
postcode = pd.read_csv('input/de_tso_postcode_gps.csv',
                       sep=';',
                       header=0)

# Drop possible duplicates in postcodes
postcode.drop_duplicates('postcode', keep='last',inplace=True)

# Show first entries
postcode.head()

** Merge geometry information by using the postcode**

In [ ]:
# Take postcode and longitude/latitude infomations
postcode= postcode[[0,3,4]]

renewables =renewables.merge(postcode, on=['postcode'], how='left')

## 3.2 Transform Geoinformation
*(for data with already existing geoinformation)*

In this section the existing geoinformation (in UTM-format) will be transformed into latidude and longitude coordiates as a uniform standard for geoinformation. 

The BNetzA data set offers UTM Geoinformation with the columns *utm_zone (UTM-Zonenwert)*, *utm_east* and *utm_north*. Most of utm_east-values include the utm_zone-value **32** at the beginning of the number. In order to properly standardize and transform this geoinformation into latitude and longitude it is necessary to remove this utm_zone value. (For all UTM entries the utm_zone 32 is used by the BNetzA.)


|utm_zone|	 utm_east|	 utm_north| comment|
|---|---|---| ----|
|32|	413151.72|	6027467.73| proper coordinates|
|32|	**32**912159.6008|	5692423.9664| caused error by 32|


**How many different utm_zone values are in the data set?**

In [ ]:
renewables.groupby(['utm_zone'])['utm_zone'].count()

**Remove the utm_zone "32" from the utm_east value**

In [ ]:
# Find entries with 32 value at the beginning
ix_32 = (renewables['utm_east'].astype(str).str[:2]=='32')
ix_notnull= renewables['utm_east'].notnull()

# Remove 32 from utm_east entries
renewables.loc[ix_32, 'utm_east'] = renewables.loc[ix_32, 'utm_east'].astype(str).str[2:].astype(float)

**Conversion UTM to lat/lon**

In [ ]:
# Convert from UTM values to latitude and longitude coordinates
try:
    renewables['lonlat'] = renewables.loc[ix_notnull, ['utm_east', 'utm_north', 'utm_zone']].apply(
        lambda x: utm.to_latlon(x[0], x[1], x[2], 'U'),
        axis=1) \
        .astype(str)
        
except:
    renewables['lonlat'] = np.NaN
    
lat = []
lon = []

for row in renewables['lonlat']:
    try:
        # Split tuple format into the column lat and lon  
        row = row.lstrip('(').rstrip(')')
        lat.append(row.split(',')[0])
        lon.append(row.split(',')[1])
    except:
        # set NAN 
        lat.append(np.NaN)
        lon.append(np.NaN)
          

renewables['latitude'] = lat
renewables['longitude'] = lon    

# Add new values to data frame lon and lat
renewables['lon'] = renewables[['longitude', 'lon']].apply(
    lambda x: x[1] if pd.isnull(x[0]) else x[0],
    axis=1)

renewables['lat'] = renewables[['latitude', 'lat']].apply(
    lambda x: x[1] if pd.isnull(x[0]) else x[0],
    axis=1)

**Check: missing coordinates by source and type**

In [ ]:
print('Missing Coordinates ',renewables.lat.isnull().sum())

renewables[renewables.lat.isnull()].groupby(['generation_type',
                                             'source']
                                            )['source'].count()

In [ ]:
renewables.info()

 # Save Data Frame
 
 Finally the merged data frame will be saved as SQLite file

In [ ]:
renewables.to_sql('raw_data_output', sqlite3.connect('raw_data.sqlite'), if_exists="replace")

---
The check, validation of the renewable power plants list and the Creation of CSV/XLSX/SQLite files can be found in Part 2 of this script. It also generates a daily timeseries of cumulated installed capacities by generation types.